In [33]:
import os
import openai
from scipy.spatial import distance
import plotly.express as px
from sklearn.cluster import KMeans
import json
import re
from tqdm import tqdm

In [4]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
import pandas as pd

concepts_syn_df = pd.read_csv("concept_synonyms_CVsubset.csv")
concepts_syn_df.head()

,general_ancestor_concept_name,general_ancestor_concept_id,descendant_concept_id,concept_name,rc,dbc,drc,ddbc,max_levels_of_separation,concept_id,concept_synonym_name
0,Disorder of cardiovascular system,134057,443784,Vascular disorder,186458,13,1408753835,22,1,443784,Angiopathy
1,Disorder of cardiovascular system,134057,443784,Vascular disorder,186458,13,1408753835,22,1,443784,Disorder of blood vessel
2,Disorder of cardiovascular system,134057,443784,Vascular disorder,186458,13,1408753835,22,1,443784,Vascular disease
3,Disorder of cardiovascular system,134057,443784,Vascular disorder,186458,13,1408753835,22,1,443784,Disorder of blood vessel (disorder)
4,Disorder of cardiovascular system,134057,44784217,Cardiac arrhythmia,84417590,19,809155077,22,2,44784217,Arrhythmia


# GPT Term Subsumption Task Start

In [ ]:
# first, determine highest level nodes/groupings

In [7]:
# Function to split a long list into smaller chunks
def chunk_list(lst, chunk_size=25):
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]
        
def generate_api_messages(term_list, chunk_size=25):
    """
    Generate a list of API message objects with chunked term lists and corresponding responses.
    
    Args:
        term_list (list): The full list of terms to be chunked.
        chunk_size (int): The maximum number of terms per chunk.
    
    Returns:
        List of dictionaries formatted for the API call.
    """
    
    # Calculate the total number of chunks
    total_parts = (len(term_list) + chunk_size - 1) // chunk_size  # Equivalent to ceil(len(term_list) / chunk_size)
    
    # Create a list to store the message objects
    api_messages = []
    
    # Generate each chunk and corresponding message
    for i, chunk in enumerate(chunk_list(term_list, chunk_size), 1):
        chunk_str = ", ".join(chunk)
        
        # User message with chunked list
        user_message = {
            "role": "user",
            "content": f"""[start part {i}/{total_parts}] {chunk_str} [end part {i}/{total_parts}]"""
        }
        
        # Assistant message acknowledging the chunk received
        assistant_message = {
            "role": "assistant",
            "content": f"received chunk {i}/{total_parts}"
        }
        
        # Append both user and assistant messages
        api_messages.append(user_message)
        api_messages.append(assistant_message)
    
    return api_messages

In [8]:
# iteration where I only ask for a subset of terms to be used to generate json

def generate_term_subsumption_chunked(full_term_list):
    # Define chunk size (e.g., 500 terms per batch)
    chunk_size = 25
    
    # Enhanced prompt for generating a detailed description of the term
    prompt = f"""I want to generate a cardiovascular medical hierarchy using a list of terms. To do this, 
    I want to come up with the top level 'nodes' or terms for which to group my terms under. 
    
    I want you to consider all possible terms I have before coming up with the top levels of the
    medical hierarchy that all my terms will fit under. Because the 
    full list is too long, I will be providing it in chunks. 
    
    For sending you the list, I will follow this rule: 
    [start part 1/10]
    Here is list 1/10 
    [end part 1/10]
    
    Then you just answer, "received part 1/10"
    
    When I tell you "All parts sent", then you can process the data and start generating the top level terms 
    for the hierarchy I will build. 
    
    Please output your result as a list, i.e., 
    - Group 1
    - Group 2
    - Group 3..
    etc 
    
    Remember, even though you are given the chunks separately, you must consider **all terms** 
    I’ve provided in my list when coming up with your top level groups.

    """

    # Create the API messages using the term list
    api_messages = generate_api_messages(full_term_list, chunk_size)
    
    # Add the final "All parts sent" message to ensure processing starts after all chunks are received
    api_messages.append({
        "role": "user",
        "content": "All parts sent."
    })
    
    # Include the system and initial prompt messages
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system", 
                "content": """You are a helpful assistant"""
            
                
            
            },
            {
                "role": "user",
                "content": prompt
            },
            {
                "role": "assistant",
                "content": "Understood, please send the content you provided"
            },
            *api_messages  # Append the chunked messages here
        ],
        max_tokens=5000,  # Allow enough space for a comprehensive description
        temperature=0.5  # Low temperature for clearer, more focused responses
    )
    
    # Extract the full response text from the chat model response
    full_response = response.choices[0].message.content
    
    return full_response  # Return the generated description


In [9]:
sampled_terms = concepts_syn_df.sample(n=500, replace=False, random_state=42)
full_terms = concepts_syn_df['concept_name'].unique()

In [18]:
# result_chunked = generate_term_subsumption(sampled_terms['concept_name'])
# result_chunked = generate_term_subsumption_chunked(sampled_terms['concept_synonym_name'])
result_chunked = generate_term_subsumption_chunked(full_terms)


In [19]:
print(result_chunked)

Thank you for sending all the parts. Based on the comprehensive list of cardiovascular terms you've provided, here are the top-level groups that can be used to organize these terms into a cardiovascular medical hierarchy:

1. **Heart Disorders**
   - Structural Heart Disorders
   - Heart Valve Disorders
   - Cardiomyopathy
   - Heart Failure
   - Myocardial Infarction
   - Arrhythmias
   - Congenital Heart Defects

2. **Vascular Disorders**
   - Arterial Disorders
   - Venous Disorders
   - Vascular Malformations
   - Aneurysms
   - Thrombosis and Embolism

3. **Hypertension and Related Disorders**
   - Essential Hypertension
   - Secondary Hypertension
   - Hypertensive Heart Disease

4. **Cerebrovascular Disorders**
   - Stroke
   - Transient Ischemic Attack
   - Cerebral Aneurysms
   - Cerebral Arterial Disorders

5. **Peripheral Vascular Disorders**
   - Peripheral Artery Disease
   - Peripheral Venous Disease

6. **Pulmonary Circulation Disorders**
   - Pulmonary Hypertension
   -

In [20]:
GPT_top_groups = ["Heart Disorders",
                 "Vascular Disorders",
                 "Hypertension and Related Disorders",
                 "Cerebrovascular Disorders",
                 "Peripheral Vascular Disorders",
                 "Pulmonary Circulation Disorders",
                 "Infectious and Inflammatory Heart Diseases",
                 "Congenital Cardiovascular Disorders",
                 "Pregnancy-Related Cardiovascular Disorders",
                 "Cardiovascular Complications of Systemic Diseases",
                 "Cardiac and Vascular Trauma",
                 "Miscellaneous Cardiovascular Disorders"]

In [80]:
subgroups_list_names = [
    'HeartDisorders_groups',
    'VascularDisorders_groups',
    'HypertensionDisorders_groups',
    'CerebrovascularDisorders_groups',
    'PeripheralVascularDisorders_groups',
    'PulmonaryCirculationDisorders_groups',
    'InfectiousInflammatoryHeartDisorders_groups',
    'CongenitalCardiovascularDisorders_groups',
    'PregnancyCardiovascularDisorders_groups',
    'SystemicDiseasesCardiovascular_groups',
    'CardiacVascularTrauma_groups',
    'MiscCardiovascularDisorders_groups'
]

In [81]:
# create subgroups for each top group: 

# Heart Disorders
HeartDisorders_groups = [
    'Structural Heart Disorders',
    'Heart Valve Disorders',
    'Cardiomyopathy',
    'Heart Failure',
    'Myocardial Infarction',
    'Arrhythmias',
    'Congenital Heart Defects',
    'Misc Heart Disorders'
]

# Vascular Disorders
VascularDisorders_groups = [
    'Arterial Disorders',
    'Venous Disorders',
    'Vascular Malformations',
    'Aneurysms',
    'Thrombosis and Embolism',
    'Misc Vascular Disorders'
]

# Hypertension and Related Disorders
HypertensionDisorders_groups = [
    'Essential Hypertension',
    'Secondary Hypertension',
    'Hypertensive Heart Disease',
    'Misc Hypertension Disorders'
]

# Cerebrovascular Disorders
CerebrovascularDisorders_groups = [
    'Stroke',
    'Transient Ischemic Attack',
    'Cerebral Aneurysms',
    'Cerebral Arterial Disorders',
    'Misc Cerebrovascular Disorders'
]

# Peripheral Vascular Disorders
PeripheralVascularDisorders_groups = [
    'Peripheral Artery Disease',
    'Peripheral Venous Disease',
    'Misc Peripheral Vascular Disorders'
]

# Pulmonary Circulation Disorders
PulmonaryCirculationDisorders_groups = [
    'Pulmonary Hypertension',
    'Pulmonary Embolism',
    'Misc Pulmonary Circulation Disorders'
]

# Infectious and Inflammatory Heart Diseases
InfectiousInflammatoryHeartDisorders_groups = [
    'Endocarditis',
    'Myocarditis',
    'Pericarditis',
    'Rheumatic Heart Disease',
    'Misc Infectious and Inflammatory Heart Diseases'
]

# Congenital Cardiovascular Disorders
CongenitalCardiovascularDisorders_groups = [
    'Congenital Heart Defects',
    'Congenital Vascular Anomalies',
    'Misc Congenital Cardiovascular Disorders'
]

# Pregnancy-Related Cardiovascular Disorders
PregnancyCardiovascularDisorders_groups = [
    'Hypertension in Pregnancy',
    'Cardiovascular Complications in Pregnancy',
    'Misc Pregnancy-Related Cardiovascular Disorders'
]

# Cardiovascular Complications of Systemic Diseases
SystemicDiseasesCardiovascular_groups = [
    'Diabetes-Related Vascular Disorders',
    'Autoimmune Vasculitis',
    'Cardiovascular Manifestations of Infectious Diseases',
    'Misc Cardiovascular Complications of Systemic Diseases'
]

# Cardiac and Vascular Trauma
CardiacVascularTrauma_groups = [
    'Traumatic Injury to Heart and Vessels',
    'Misc Cardiac and Vascular Trauma'
]

# Miscellaneous Cardiovascular Disorders
MiscCardiovascularDisorders_groups = [
    'Cardiac Tumors',
    'Other Rare Cardiovascular Conditions',
    'Misc Miscellaneous Cardiovascular Disorders'
]



In [83]:
# Create a dictionary to store each group dataframe
subgroup_names = {}

# Split the dataframe based on the 'Group' column
for i in range(0, len(GPT_top_groups)):
    subgroup_names[GPT_top_groups[i]] = eval(subgroups_list_names[i])

In [84]:
subgroup_names

{'Heart Disorders': ['Structural Heart Disorders',
  'Heart Valve Disorders',
  'Cardiomyopathy',
  'Heart Failure',
  'Myocardial Infarction',
  'Arrhythmias',
  'Congenital Heart Defects',
  'Misc Heart Disorders'],
 'Vascular Disorders': ['Arterial Disorders',
  'Venous Disorders',
  'Vascular Malformations',
  'Aneurysms',
  'Thrombosis and Embolism',
  'Misc Vascular Disorders'],
 'Hypertension and Related Disorders': ['Essential Hypertension',
  'Secondary Hypertension',
  'Hypertensive Heart Disease',
  'Misc Hypertension Disorders'],
 'Cerebrovascular Disorders': ['Stroke',
  'Transient Ischemic Attack',
  'Cerebral Aneurysms',
  'Cerebral Arterial Disorders',
  'Misc Cerebrovascular Disorders'],
 'Peripheral Vascular Disorders': ['Peripheral Artery Disease',
  'Peripheral Venous Disease',
  'Misc Peripheral Vascular Disorders'],
 'Pulmonary Circulation Disorders': ['Pulmonary Hypertension',
  'Pulmonary Embolism',
  'Misc Pulmonary Circulation Disorders'],
 'Infectious and Inf

# Determine top level groups

In [41]:
def generate_term_grouping(term, ancestor_list):

    # Construct the prompt for the current chunk
    prompt = f"""Please place this term: "{term}" into ONE of these categories: {ancestor_list}. 

    output only the category. do not output anything else. 

    """

    # Call the API for each chunk
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system", 
                "content": """you are a helpful assistant."""
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        max_tokens=100,  # Adjust if necessary for your use case
        temperature=0.1  # Low temperature for clearer, more focused responses
    )

    # Extract the full response text from the chat model response
    full_response = response.choices[0].message.content
    
    return full_response  # Return the combined response or process it further

In [42]:
group_list = []
for term in tqdm(full_terms):
    group_list.append(generate_term_grouping(term, GPT_top_groups))

100%|█████████████████████████████████████████| 1533/1533 [11:34<00:00,  2.21it/s]


In [109]:
full_terms

array(['Vascular disorder', 'Cardiac arrhythmia',
       'Supraventricular arrhythmia', ...,
       'Congenital stenosis of pulmonary veins', 'Erythrocyanosis',
       'Accelerated coronary artery disease in transplanted heart'],
      dtype=object)

In [54]:
# this will give the top level groups for each term
df_toplevel = pd.DataFrame(
    {'Term': full_terms,
     'Group': group_list
    })

df_toplevel.head()

,Term,Group
0,Vascular disorder,Vascular Disorders
1,Cardiac arrhythmia,Heart Disorders
2,Supraventricular arrhythmia,Heart Disorders
3,Heart failure,Heart Disorders
4,Atrial arrhythmia,Heart Disorders


In [55]:
df_toplevel['Group'].unique()

array(['Vascular Disorders', 'Heart Disorders',
       'Hypertension and Related Disorders',
       'Miscellaneous Cardiovascular Disorders',
       "'Miscellaneous Cardiovascular Disorders'",
       'Cerebrovascular Disorders', 'Peripheral Vascular Disorders',
       'Pulmonary Circulation Disorders',
       'Congenital Cardiovascular Disorders',
       'Pregnancy-Related Cardiovascular Disorders',
       'Infectious and Inflammatory Heart Diseases',
       'Cardiac and Vascular Trauma', "'Vascular Disorders'",
       'Cardiovascular Complications of Systemic Diseases',
       "'Hypertension and Related Disorders'",
       "'Pregnancy-Related Cardiovascular Disorders'",
       "'Heart Disorders'",
       "'Cardiovascular Complications of Systemic Diseases'",
       "'Peripheral Vascular Disorders'",
       "'Congenital Cardiovascular Disorders'"], dtype=object)

In [ ]:
#df cleanup 
df_toplevel['Group'] = df_toplevel['Group'].str.replace(r"^'|'$", "", regex=True)

# Verify the unique values after cleaning
unique_values = df_toplevel['Group'].unique()
print(unique_values)

In [62]:
# Count the number of rows for each group
group_counts = df_toplevel.groupby('Group').size().reset_index(name='Count')

# Display the result
print(group_counts)

                                                Group  Count
0                         Cardiac and Vascular Trauma    165
1   Cardiovascular Complications of Systemic Diseases     17
2                           Cerebrovascular Disorders    116
3                 Congenital Cardiovascular Disorders    107
4                                     Heart Disorders    376
5                  Hypertension and Related Disorders     44
6          Infectious and Inflammatory Heart Diseases     92
7              Miscellaneous Cardiovascular Disorders     73
8                       Peripheral Vascular Disorders    119
9          Pregnancy-Related Cardiovascular Disorders    108
10                    Pulmonary Circulation Disorders     38
11                                 Vascular Disorders    278


In [73]:
# split up df into more dfs

# Get the unique groups
# this order matters bc it has to be the same order as subgroups_list_names
unique_groups = ['Heart Disorders', 'Vascular Disorders', 
       'Hypertension and Related Disorders',
       'Cerebrovascular Disorders','Peripheral Vascular Disorders',
       'Pulmonary Circulation Disorders',
       'Infectious and Inflammatory Heart Diseases',
       'Congenital Cardiovascular Disorders',
       'Pregnancy-Related Cardiovascular Disorders',
       'Cardiovascular Complications of Systemic Diseases',
       'Cardiac and Vascular Trauma',
       'Miscellaneous Cardiovascular Disorders']

# Create a dictionary to store each group dataframe
group_dfs = {}

# Split the dataframe based on the 'Group' column
for group in unique_groups:
    group_dfs[group] = df_toplevel[df_toplevel['Group'] == group]

In [74]:
unique_groups

['Heart Disorders',
 'Vascular Disorders',
 'Hypertension and Related Disorders',
 'Cerebrovascular Disorders',
 'Peripheral Vascular Disorders',
 'Pulmonary Circulation Disorders',
 'Infectious and Inflammatory Heart Diseases',
 'Congenital Cardiovascular Disorders',
 'Pregnancy-Related Cardiovascular Disorders',
 'Cardiovascular Complications of Systemic Diseases',
 'Cardiac and Vascular Trauma',
 'Miscellaneous Cardiovascular Disorders']

In [75]:
group_dfs[GPT_top_groups[0]]

,Term,Group
1,Cardiac arrhythmia,Heart Disorders
2,Supraventricular arrhythmia,Heart Disorders
3,Heart failure,Heart Disorders
4,Atrial arrhythmia,Heart Disorders
5,Fibrillation,Heart Disorders
...,...,...
1524,Interventricular cardiac septal hypertrophy,Heart Disorders
1525,Exertional hypotension,Heart Disorders
1527,Hypertrophic cardiomegaly,Heart Disorders
1529,Primary idiopathic hypertrophic cardiomyopathy,Heart Disorders


# next level subsumption groups

In [137]:
groupings_top_groups = {} # empty dictionary to store results

for group in unique_groups: # for each top level group
    print(f"starting {group}")
    res = []
    # print(group)
    terms_in_topgroup = group_dfs[group]['Term'] # these are the terms in the top level group
    for term in tqdm(terms_in_topgroup):
        # print(f"starting {group}")
        res.append(generate_term_grouping(term, subgroup_names[group]))
        
    # make it into a temp df
    df_temp = pd.DataFrame(
        {'Term': terms_in_topgroup,
         'Group': res
        })
    df_temp['Group'] = df_temp['Group'].str.replace(r"^'|'$", "", regex=True)
    df_temp.to_csv(group + ".csv", index = False) # ensures that we save the CSV

    # add temp df into dictionary
    groupings_top_groups[group] = df_temp

starting Heart Disorders


100%|███████████████████████████████████████████| 376/376 [03:01<00:00,  2.07it/s]


starting Vascular Disorders


100%|███████████████████████████████████████████| 278/278 [01:56<00:00,  2.39it/s]


starting Hypertension and Related Disorders


...

100%|█████████████████████████████████████████████| 17/17 [00:08<00:00,  2.11it/s]


starting Cardiac and Vascular Trauma


100%|███████████████████████████████████████████| 165/165 [01:28<00:00,  1.87it/s]


starting Miscellaneous Cardiovascular Disorders


100%|█████████████████████████████████████████████| 73/73 [00:35<00:00,  2.03it/s]


In [ ]:
groupings_top_groups = {}

# reading in dfs
for group in unique_groups: 
    groupings_top_groups[group] = pd.read_csv(group + ".csv")

In [308]:
# Count the number of rows for each group

for group in unique_groups: 
    subgroup_counts = groupings_top_groups[group].groupby('Group').size().reset_index(name='Count')

    # Display the result
    print(subgroup_counts)

                        Group  Count
0                 Arrhythmias     95
1              Cardiomyopathy     35
2               Heart Failure     49
3       Heart Valve Disorders     76
4        Misc Heart Disorders     50
5       Myocardial Infarction     46
6  Structural Heart Disorders     25
                     Group  Count
0                Aneurysms     29
1       Arterial Disorders     47
2  Misc Vascular Disorders     62
3  Thrombosis and Embolism     68
4   Vascular Malformations     29
5         Venous Disorders     43
                         Group  Count
0       Essential Hypertension      5
1   Hypertensive Heart Disease     18
2  Misc Hypertension Disorders     11
3       Secondary Hypertension     10
                            Group  Count
0              Cerebral Aneurysms      2
1     Cerebral Arterial Disorders     42
2  Misc Cerebrovascular Disorders     39
3                          Stroke     26
4       Transient Ischemic Attack      7
                              

# now determine the hierarchy within each subtree

In [164]:
def generate_term_hierarchy(branch_list, subgroup, group):

    # Construct the prompt for the current chunk
    prompt = f"""I am generating a cardiovascular diseases term hierarchy. 
    I have already generated top level groups and subgroups. The top level group I am working on right now
    is called "{group}", and "{subgroup}" is a node within that group. 
    
    I would like to organize the leaves within this subgroup. 
    Please organize the following terms into a hierarchy. 
    
    Put it in this format. If necessary, you can have more depth than this example here.
    1. {group}
    1.1 {subgroup}
    1.1.1 organize the terms here
    1.1.1.1 child term of term 1.1.1
    1.1.2 sibling term to 1.1.1
    
    Then, output the json format of that hierarchy.
    
    Here are the terms: {branch_list}

    output only the hierarchy, both in list form and json. do not output anything else. 

    """

    # Call the API for each chunk
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system", 
                "content": """you are a helpful assistant."""
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        max_tokens=10000,  # Adjust if necessary for your use case
        temperature=0.1  # Low temperature for clearer, more focused responses
    )

    # Extract the full response text from the chat model response
    full_response = response.choices[0].message.content
    
    return full_response  # Return the combined response or process it further

In [140]:
groupings_top_groups[group]

,Term,Group
9,Acute disease of cardiovascular system,Other Rare Cardiovascular Conditions
18,Chronic disease of cardiovascular system,Misc Miscellaneous Cardiovascular Disorders
30,Migraine,Misc Miscellaneous Cardiovascular Disorders
81,Migraine without aura,Misc Miscellaneous Cardiovascular Disorders
131,Refractory migraine,Misc Miscellaneous Cardiovascular Disorders
...,...,...
1458,Thrombophlebitis of breast,Misc Miscellaneous Cardiovascular Disorders
1464,Kartagener syndrome,Misc Miscellaneous Cardiovascular Disorders
1496,Cogan's syndrome,Other Rare Cardiovascular Conditions
1507,Fahr's syndrome,Misc Miscellaneous Cardiovascular Disorders


In [146]:
# current_branch = one of the main groups
current_branch_df = groupings_top_groups[group] # df of terms and subgroups, all in the main group
current_subgroup = subgroup_names[group][0] # this is a string
terms = current_branch_df[current_branch_df['Group'] == current_subgroup]['Term']

In [151]:
print(generate_term_hierarchy(terms, current_subgroup, group))

1. Miscellaneous Cardiovascular Disorders
    1.1 Cardiac Tumors
        1.1.1 Malignant tumor of heart
            1.1.1.1 Primary malignant neoplasm of heart
        1.1.2 Neoplasm of pericardium

```json
{
    "Miscellaneous Cardiovascular Disorders": {
        "Cardiac Tumors": {
            "Malignant tumor of heart": {
                "Primary malignant neoplasm of heart": {}
            },
            "Neoplasm of pericardium": {}
        }
    }
}
```


In [198]:
# first within each group, loop over each subgroup

hierarchy_text_output = [] #save it as a list
groups_for_df = []
subgroups_for_df = []

for group in unique_groups: # first loop over each of the main groups
    current_branch_df = groupings_top_groups[group] # groupings_top_groups is a dictionary of group dfs
    print(group)

    for subgroup in tqdm(subgroup_names[group]):
        print(subgroup)
        
        terms = current_branch_df[current_branch_df['Group'] == subgroup]['Term']
        # print(len(terms))
        
        subgroups_for_df.append(subgroup)
        groups_for_df.append(group)
        res = generate_term_hierarchy(terms, subgroup, group)
        hierarchy_text_output.append(res)
        print(res)

Heart Disorders


  0%|                                                         | 0/8 [00:00<?, ?it/s]

Structural Heart Disorders


 12%|██████▏                                          | 1/8 [00:06<00:46,  6.58s/it]

1. Heart Disorders
1.1 Structural Heart Disorders
1.1.1 Ventricular Disorders
1.1.1.1 Ventricular hypertrophy
1.1.1.1.1 Left ventricular hypertrophy
1.1.1.1.2 Right ventricular hypertrophy
1.1.1.2 Ventricular dilatation
1.1.1.2.1 Right cardiac ventricular dilatation
1.1.1.3 Ventricular aneurysm
1.1.1.3.1 Left ventricular aneurysm
1.1.1.4 Subaortic stenosis
1.1.1.5 Left ventricular outflow tract obstruction
1.1.1.6 Post-infarction ventricular septal defect
1.1.2 Atrial Disorders
1.1.2.1 Atrial septal defect due to and following acute myocardial infarction
1.1.2.2 Atrial dilatation
1.1.2.2.1 Right atrial dilatation
1.1.2.3 Atrial enlargement
1.1.2.3.1 Left atrial enlargement
1.1.2.3.2 Bilateral enlargement of atria
1.1.2.4 Atrial aneurysm
1.1.2.5 Atrial myxoma
1.1.3 Coronary Artery Disorders
1.1.3.1 Calcific coronary arteriosclerosis
1.1.3.2 Acquired coronary artery fistula
1.1.4 Hypertensive heart disease
1.1.5 Cardiomegaly
1.1.6 Right ventricular abnormality
1.1.7 Papillary muscle diso

...

1. Miscellaneous Cardiovascular Disorders
    1.1 Cardiac Tumors
        1.1.1 Neoplasm of pericardium
        1.1.2 Malignant tumor of heart
            1.1.2.1 Primary malignant neoplasm of heart

```json
{
    "Miscellaneous Cardiovascular Disorders": {
        "Cardiac Tumors": {
            "Neoplasm of pericardium": {},
            "Malignant tumor of heart": {
                "Primary malignant neoplasm of heart": {}
            }
        }
    }
}
```
Other Rare Cardiovascular Conditions


 67%|████████████████████████████████▋                | 2/3 [00:07<00:04,  4.05s/it]

```
1. Miscellaneous Cardiovascular Disorders
1.1 Other Rare Cardiovascular Conditions
1.1.1 Acute disease of cardiovascular system
1.1.2 Idiopathic hypotension
1.1.3 Carotid sinus disorders
1.1.3.1 Carotid sinus hypersensitivity
1.1.3.2 Carotid sinus syncope
1.1.4 Kyphoscoliotic heart disease
1.1.5 Portal pyemia
1.1.6 Postural orthostatic tachycardia syndrome
1.1.7 Heart-lung transplant complications
1.1.7.1 Heart-lung transplant failure and rejection
1.1.8 Pericardial and pleuropericardial cysts
1.1.8.1 Pericardial cyst
1.1.8.2 Pleuropericardial cyst
1.1.9 Cogan's syndrome
1.1.10 Athlete's heart
```

```json
{
  "Miscellaneous Cardiovascular Disorders": {
    "Other Rare Cardiovascular Conditions": {
      "Acute disease of cardiovascular system": {},
      "Idiopathic hypotension": {},
      "Carotid sinus disorders": {
        "Carotid sinus hypersensitivity": {},
        "Carotid sinus syncope": {}
      },
      "Kyphoscoliotic heart disease": {},
      "Portal pyemia": {},
     

100%|█████████████████████████████████████████████████| 3/3 [00:26<00:00,  8.81s/it]

1. Miscellaneous Cardiovascular Disorders
    1.1 Misc Miscellaneous Cardiovascular Disorders
        1.1.1 Chronic disease of cardiovascular system
        1.1.2 Migraine
            1.1.2.1 Migraine without aura
                1.1.2.1.1 Migraine without aura, not refractory
            1.1.2.2 Refractory migraine
                1.1.2.2.1 Refractory migraine without aura
                1.1.2.2.2 Refractory migraine variants
            1.1.2.3 Status migrainosus
            1.1.2.4 Transformed migraine
            1.1.2.5 Migraine variants
                1.1.2.5.1 Migraine variants, not intractable
            1.1.2.6 Menstrual migraine
            1.1.2.7 Menstrual status migrainosus
            1.1.2.8 Ophthalmoplegic migraine
            1.1.2.9 Ophthalmic migraine
            1.1.2.10 Abdominal migraine
            1.1.2.11 Hemicrania continua
            1.1.2.12 Chronic mixed headache syndrome
        1.1.3 Cardiovascular system complication of procedure
        1.1.4 Cardia

In [199]:
json_df = pd.DataFrame(
        {'Group': groups_for_df,
         'Subgroup': subgroups_for_df,
         'Text_output': hierarchy_text_output
        })

json_df.to_csv("Raw subsumption output pass all.csv", index = False) # ensures that we save the CSV

# combine json files

In [256]:
import re

# Initialize an empty list to store the data
json_data = []

# for text_output in hierarchy_text_output: 
for text_output in json_df["Text_output"]:
    chunk = re.findall(r'```json\n(.*?)\n```', text_output, re.DOTALL)
    # print(chunk[0:50])
    try:
        # Parse the JSON string into a Python dictionary
        json_obj = json.loads(chunk[0])
        json_data.append(json_obj)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        print("Attempting manual cleanup of chunk:", chunk[:100])  # Print first 100 characters for debugging
        
    



In [257]:
# Step 2: Merge all dictionaries in list
merged_dict = {}
for pdict in json_data:
    for key, value in pdict.items():
        if key in merged_dict:
            # Merge the nested dictionaries if the key already exists
            merged_dict[key].update(value)
        else:
            merged_dict[key] = value

# Step 3: Write the merged dictionary to a JSON file
with open("merged_output_CVraw.json", "w") as outfile:
    json.dump(merged_dict, outfile, indent=4)

print("Merged JSON saved to merged_output_CVraw.json")

Merged JSON saved to merged_output_CVraw.json


# extract relationships

In [258]:
with open("merged_output_CVraw.json") as json_file:
    json_data = json.load(json_file)
    # print(json_data)

In [259]:
def traverse_hierarchy(node, current_term=None):
    """
    Recursively traverse the hierarchy.
    Returns a dictionary with:
    {
        "term": <the current term or None if top>,
        "descendants": { "descendant_term": separation_distance, ... }
    }

    If 'node' is a dict:
      - each key is a sub-term
      - value can be a dict or a list
    If 'node' is a list:
      - each element is a leaf term (string)
    """
    descendants = {}
    # If current_term is None, that means we are at the top (like "Cardiovascular")
    # But top level keys are also considered terms.
    # We'll handle the top by checking if node is dict and has only one key, that key will be current_term.

    # If node is a dictionary and current_term is None, we might be at the top-level structure:
    if current_term is None and isinstance(node, dict) and len(node) == 1:
        # Top-level single key
        top_key = list(node.keys())[0]
        # Recurse into that key
        result = traverse_hierarchy(node[top_key], current_term=top_key)
        # Include the top-level term with separation=0
        result["descendants"][top_key] = 0
        return result

    if isinstance(node, dict):
        # We are at a node that is a dictionary. The keys are sub-terms.
        # For each key, recurse:
        for k, v in node.items():
            child_result = traverse_hierarchy(v, current_term=k)
            # Now child_result["descendants"] includes k->k(0), and all descendants
            # For the current node, k is a direct descendant (distance=1),
            # and all k's descendants are +1 from that.
            # Merge them into this node's descendants.
            for d_term, dist in child_result["descendants"].items():
                # since from current_term perspective, k is one level below current_term,
                # so we add +1 to their distances
                descendants[d_term] = min(descendants.get(d_term, float('inf')), dist + 1)
        
        # Don't forget to include the current_term itself with separation 0
        # if current_term is not None
        if current_term is not None:
            descendants[current_term] = 0
        return {"term": current_term, "descendants": descendants}

    elif isinstance(node, list):
        # This is a leaf-level list of terms. All are direct descendants of current_term.
        for item in node:
            # item is a leaf
            # Distance from current_term to item = 1
            descendants[item] = 1
        # Include current_term itself if not None
        if current_term is not None:
            descendants[current_term] = 0
        return {"term": current_term, "descendants": descendants}

    else:
        # It's neither dict nor list; unexpected structure
        # Just return current_term itself
        if current_term is not None:
            return {"term": current_term, "descendants": {current_term: 0}}
        else:
            return {"term": current_term, "descendants": {}}


# Run the traversal starting from the top
result = traverse_hierarchy(json_data)


all_relationships = []  # will hold tuples of (ancestor, descendant, separation)

def extract_relationships(struct, ancestors=None):
    """
    Recursively go through the structure and record:
    (ancestor_term, descendant_term, separation)
    including ancestor=descendant (0 separation).
    
    ancestors is a list of (term, depth).
    depth increases as we go down.
    """
    if ancestors is None:
        ancestors = []
        
    # If struct is a dict: keys are terms, values can be dict or list
    if isinstance(struct, dict):
        # If top-level and has one key, start from there
        if len(ancestors) == 0 and len(struct) == 1:
            top_key = list(struct.keys())[0]
            # Add top_key as ancestor
            current_ancestors = ancestors + [(top_key, 0)]
            # ancestor=descendant with 0 separation
            all_relationships.append((top_key, top_key, 0))
            extract_relationships(struct[top_key], current_ancestors)
        else:
            # We are inside a dictionary that represents multiple children
            for k, v in struct.items():
                # The new node is a child of whatever the last ancestor in 'ancestors' is
                # If ancestors is empty, this might be top-level terms
                base_depth = ancestors[-1][1] if ancestors else 0
                current_ancestors = ancestors + [(k, base_depth+1)]
                # Record k->k with 0 separation
                all_relationships.append((k, k, 0))
                
                # Also record ancestor->k relationships
                for anc, anc_depth in ancestors:
                    separation = (base_depth+1 - anc_depth)
                    all_relationships.append((anc, k, separation))
                
                extract_relationships(v, current_ancestors)
                
    elif isinstance(struct, list):
        # We are at a leaf list. Each item is a leaf term.
        base_depth = ancestors[-1][1] if ancestors else 0
        for item in struct:
            # item is a descendant
            # item->item (0 separation)
            all_relationships.append((item, item, 0))
            # Also record ancestors->item
            for anc, anc_depth in ancestors:
                separation = (base_depth+1 - anc_depth)
                all_relationships.append((anc, item, separation))

    else:
        # It's a string or something unexpected; just consider it a term by itself
        # This would be unusual given the structure we have
        pass

In [260]:
json_data

{'Heart Disorders': {'Structural Heart Disorders': {'Ventricular Disorders': {'Ventricular hypertrophy': {'Left ventricular hypertrophy': {},
     'Right ventricular hypertrophy': {}},
    'Ventricular dilatation': {'Right cardiac ventricular dilatation': {}},
    'Ventricular aneurysm': {'Left ventricular aneurysm': {}},
    'Subaortic stenosis': {},
    'Left ventricular outflow tract obstruction': {},
    'Post-infarction ventricular septal defect': {}},
   'Atrial Disorders': {'Atrial septal defect due to and following acute myocardial infarction': {},
    'Atrial dilatation': {'Right atrial dilatation': {}},
    'Atrial enlargement': {'Left atrial enlargement': {},
     'Bilateral enlargement of atria': {}},
    'Atrial aneurysm': {},
    'Atrial myxoma': {}},
   'Coronary Artery Disorders': {'Calcific coronary arteriosclerosis': {},
    'Acquired coronary artery fistula': {}},
   'Hypertensive heart disease': {},
   'Cardiomegaly': {},
   'Right ventricular abnormality': {},
   '

In [261]:
# Clear and run the relationship extraction
all_relationships.clear()
extract_relationships(json_data)

# Now all_relationships should have all ancestor/descendant pairs from every node's perspective.

# Let's put it into a nice table form (e.g. list of dicts)
table = []
# Use a set to avoid duplicates if needed
seen = set()
for ancestor, descendant, sep in all_relationships:
    # (ancestor, descendant, sep)
    if (ancestor, descendant, sep) not in seen:
        seen.add((ancestor, descendant, sep))
        table.append({
            "ancestor_term": ancestor,
            "descendant_term": descendant,
            "min_levels_of_separation": sep
        })

# Sort table by ancestor_term, then descendant_term, then sep for cleanliness (optional)
table.sort(key=lambda x: (x["ancestor_term"], x["descendant_term"], x["min_levels_of_separation"]))

# Print some sample rows
# for row in table[:50]:
#     print(row)

In [262]:
# save as CSV
df_relationships = pd.DataFrame(table)
df_relationships.head()

df_relationships.to_csv("concept_subsumption_GPTresult_v2.csv", index=False)

In [263]:
df_relationships.iloc[55:71]


,ancestor_term,descendant_term,min_levels_of_separation
55,Acute Myocardial Infarction,Acute ST Segment Elevation Myocardial Infarction,1
56,Acute Myocardial Infarction,Acute ST Segment Elevation Myocardial Infarcti...,2
57,Acute Myocardial Infarction,Acute ST Segment Elevation Myocardial Infarcti...,2
58,Acute Myocardial Infarction,Acute ST Segment Elevation Myocardial Infarcti...,2
59,Acute Myocardial Infarction,Acute ST Segment Elevation Myocardial Infarcti...,2
60,Acute Myocardial Infarction,Acute ST Segment Elevation Myocardial Infarcti...,2
61,Acute Myocardial Infarction,Acute Subendocardial Infarction,1
62,Acute Myocardial Infarction,Non-Q Wave Myocardial Infarction,2
63,Acute Myocardial Infarction,True Posterior Myocardial Infarction,2
64,Acute Myocardial Infarction Due to Right Coron...,Acute Myocardial Infarction Due to Right Coron...,0


In [264]:
df_relationships

,ancestor_term,descendant_term,min_levels_of_separation
0,AV node arrhythmia,AV node arrhythmia,0
1,Abdominal angina,Abdominal angina,0
2,Abdominal aortic ectasia,Abdominal aortic ectasia,0
3,Abdominal migraine,Abdominal migraine,0
4,Abnormal fetal heart beat first noted during l...,Abnormal fetal heart beat first noted during l...,0
...,...,...,...
4576,Visceral Artery Aneurysms,Aneurysm of visceral artery,1
4577,Visceral Artery Aneurysms,Visceral Artery Aneurysms,0
4578,Vulval varices,Genital varices in pregnancy,1
4579,Vulval varices,Vulval varices,0


In [307]:
len(df_relationships["descendant_term"].unique())

1214

In [294]:
df_relationships["ancestor_term"].values

array(['AV node arrhythmia', 'Abdominal angina',
       'Abdominal aortic ectasia', ..., 'Vulval varices',
       'Vulval varices', 'Williams syndrome'], dtype=object)

In [291]:
len(df_relationships["descendant_term"].unique())

1214

In [305]:
abc = [i.lower() for i in df_relationships["descendant_term"].unique()]
bcd = [i.lower() for i in full_terms]

len(set(abc).intersection(set(bcd)))

1023

In [306]:
len(full_terms)

1533

# Evaluation

In [295]:
# make table from our data

# Filter to get only those rows where level of separation > 1
valid_subsumptions = df_relationships[df_relationships['min_levels_of_separation'] > 0].copy()

# filter to only valid terms
#valid_subsumptions = valid_subsumptions[
#    valid_subsumptions['ancestor_term'].isin(full_terms)
#]

# Convert all terms to lowercase for case insensitivity
valid_subsumptions['ancestor_term'] = valid_subsumptions['ancestor_term'].str.lower()
valid_subsumptions['descendant_term'] = valid_subsumptions['descendant_term'].str.lower()

# Create the final dataframe with just 'ancestor' and 'descendant' columns
final_df = valid_subsumptions[['ancestor_term', 'descendant_term']].rename(columns={
    'ancestor_term': 'ancestor',
    'descendant_term': 'descendant'
})

print(final_df.head())


                                        ancestor  \
8                     abnormality of aortic arch   
9                     abnormality of aortic arch   
10                    abnormality of aortic arch   
13  abnormality of common atrioventricular valve   
15                  abnormality of systemic vein   

                                           descendant  
8                   abnormality of aortic arch branch  
9   congenital abnormality of supraaortic branch o...  
10                  congenital anomaly of aortic arch  
13  ebstein's anomaly of common atrioventricular v...  
15                     internal jugular vein stenosis  


In [296]:
final_df

,ancestor,descendant
8,abnormality of aortic arch,abnormality of aortic arch branch
9,abnormality of aortic arch,congenital abnormality of supraaortic branch o...
10,abnormality of aortic arch,congenital anomaly of aortic arch
13,abnormality of common atrioventricular valve,ebstein's anomaly of common atrioventricular v...
15,abnormality of systemic vein,internal jugular vein stenosis
...,...,...
4571,viral myocarditis,viral carditis
4574,visceral artery aneurysms,aneurysm of renal artery
4575,visceral artery aneurysms,aneurysm of splenic artery
4576,visceral artery aneurysms,aneurysm of visceral artery


In [270]:
final_df.to_csv("valid_subsumptions_GPTresult_v2.csv", index=False)

In [297]:
gold_standard = pd.read_csv("concept_subsumption_CVsubset.csv")
gold_standard.head()

,descendant_concept_id,descendant_concept_name,ancestor_concept_id,ancestor_concept_name,min_levels_of_separation
0,22340,Esophageal varices without bleeding,22340,Esophageal varices without bleeding,0
1,198467,Acute gastric ulcer with hemorrhage and obstru...,23237,Acute peptic ulcer with hemorrhage AND obstruc...,1
2,316457,Mallory-Weiss syndrome,23245,Esophageal bleeding,1
3,438881,Closed fracture of C1-C4 level with anterior c...,24818,Injury of neck,5
4,438903,Injury of external jugular vein,24818,Injury of neck,2


In [299]:
# Filter to get only those rows where level of separation > 1
gold_standard_subsumptions = gold_standard[gold_standard['min_levels_of_separation'] >= 1].copy()

gold_standard_subsumptions = gold_standard_subsumptions[
    gold_standard_subsumptions['ancestor_concept_name'].isin(full_terms) & 
    gold_standard_subsumptions['descendant_concept_name'].isin(full_terms)
]

# Convert all terms to lowercase for case insensitivity
gold_standard_subsumptions['ancestor_concept_name'] = gold_standard_subsumptions['ancestor_concept_name'].str.lower()
gold_standard_subsumptions['descendant_concept_name'] = gold_standard_subsumptions['descendant_concept_name'].str.lower()

# Create the final dataframe with just 'ancestor' and 'descendant' columns
gold_final_df = gold_standard_subsumptions[['ancestor_concept_name', 'descendant_concept_name']].rename(columns={
    'ancestor_concept_name': 'ancestor',
    'descendant_concept_name': 'descendant'
})

print(gold_final_df.head())


                     ancestor  \
15         esophageal varices   
16         esophageal varices   
17         esophageal varices   
57   injury of saphenous vein   
132      injury of iliac vein   

                                            descendant  
15   esophageal varices associated with another dis...  
16                 esophageal varices without bleeding  
17                         bleeding esophageal varices  
57                       greater saphenous vein injury  
132                      injury of internal iliac vein  


In [300]:
gold_final_df.to_csv("gold_standard_subsumptions_CV.csv", index=False)

In [303]:
# Convert to sets of tuples (ancestor, descendant)
predicted_set = set(zip(final_df['ancestor'], final_df['descendant']))
gold_set = set(zip(gold_final_df['ancestor'], gold_final_df['descendant']))


# filter so that predicted and gold are valid sets (terms are present in full_terms)
# Convert full_terms to lowercase and make it a set for O(1) lookups
valid_terms = set(t.lower() for t in full_terms)

# Now filter predicted_set and gold_set so that both ancestor and descendant are in valid_terms
predicted_filtered = {(a.lower(), d.lower()) for (a, d) in predicted_set 
                      if a.lower() in valid_terms and d.lower() in valid_terms}

gold_filtered = {(a.lower(), d.lower()) for (a, d) in gold_set 
                 if a.lower() in valid_terms and d.lower() in valid_terms}


In [304]:
# Compute TP, FP, FN
TP = predicted_filtered.intersection(gold_filtered)
FP = predicted_filtered.difference(gold_filtered)
FN = gold_filtered.difference(predicted_filtered)

# Calculate precision and recall
precision = len(TP) / (len(TP) + len(FP)) if (len(TP) + len(FP)) > 0 else 0
recall = len(TP) / (len(TP) + len(FN)) if (len(TP) + len(FN)) > 0 else 0

print("Precision:", precision)
print("Recall:", recall)

Precision: 0.7432605905006419
Recall: 0.0869891826923077
